In [1]:
import rasterio
import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from rasterio.plot import show
import rasterio
import numpy as np
import requests

In [3]:
# # update the wd path to be able to laod the modules
# # os.getcwd()
os.chdir('..')

In [6]:
os.environ['GCS_BUCKET']='gee-exports'
os.environ['GOOGLE_APPLICATION_USER']='developers@citiesindicators.iam.gserviceaccount.com'
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='C:\\Users\Saif.Shabou\OneDrive - World Resources Institute\Documents\cities-indicators-framework\citymetrix\credentials-citiesindicators.json'

In [7]:
from city_metrix import (
    built_land_without_tree_cover, 
    mean_tree_cover, 
    built_land_with_low_surface_reflectivity, 
    built_land_with_high_land_surface_temperature
)


Authenticating to GEE with configured credentials file.


# Select a city

In [8]:
# Get city boundary
cities_api_url = "http://3.225.116.209:8000/cities"
city_name = "BRA-Salvador"
admin_level = "ADM4union"
city_geom = cities_api_url + "/"+city_name+"/"+admin_level+"/geojson"
city_gdf = gpd.read_file(city_geom)
city_gdf.head()

,id,geo_id,geo_name,geo_level,geo_parent_name,geo_version,geometry
0,0,BRA-Salvador_ADM4-union_1,BRA-Salvador,ADM4union,BRA-Salvador,0,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140..."


In [10]:
import coiled
cluster = coiled.Cluster(name="city_metrix", 
                         use_best_zone=True, 
                         region="eu-central-1",
                         compute_purchase_option="spot_with_fallback")


from distributed import Client
client = Client(cluster)


# set autoscaling to keep it at 1 worker when idle but up to 100 when ctive
cluster.adapt(minimum=1, maximum=100, wait_count=10, target_duration="1s", interval="1s")

Output()

Package - TBB, Cannot find TBB~=0.2.0 on pypi.org, currently only packages from pypi.org are supported. Custom 
conda channels are supported if your package is available there instead.

Package - icc_rt, icc_rt~=2019.0.0 has no install candidate for linux-64 on pkgs/main

Package - powershell_shortcut, powershell_shortcut~=0.0.1 has no install candidate for linux-64 on pkgs/main

Package - scikit-learn-intelex, Cannot find scikit-learn-intelex~=2021.20220215.102710 on pypi.org, currently only 
packages from pypi.org are supported. Custom conda channels are supported if your package is available there 
instead.

...

Package - autopep8, Pip check had the following issues that need resolving:

╭────────────────────────────────────────── Package Info ──────────────────────────────────────────╮
│                           ╷                                                                      │
│   Package                 │ Note                                                                 │
│ ╶─────────────────────────┼────────────────────────────────────────────────────────────────────╴ │
│   coiled_local_cities_cif │ Source wheel built from C:\Users\Saif.Shabou\OneDrive - World        │
│                           │ Resources                                                            │
│                           │ Institute\Documents\cities-indicators-framework\citymetrix\cities-   │
│                           │ cif                                                                  │
│                           ╵                                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────── Not Synced with Cluster ─────────────────────────────────────╮
│                        ╷                                                             ╷           │
│   Package              │ Error                                                       │ Risk      │
│ ╶──────────────────────┼─────────────────────────────────────────────────────────────┼─────────╴ │
│   xlwings              │ xlwings~=0.24.9 has no install candidate for linux-64 on    │ Warning   │
│                        │ pkgs/main                                                   │           │
│   wincertstore         │ wincertstore~=0.2.0 has no install candidate for linux-64   │ Warning   │
│                        │ on pkgs/main                                                │           │
│   werkzeug             │ Pip check had the following issues that need resolving:     │ Warning   │
│                        │ werkzeug 2.3.6 has requirement MarkupSafe>=2.1.1, but you   │           │
│                        │ have markupsafe 2.0.1.                                      │           │
│   scikit-learn-intelex │ Cannot find scikit-learn-intelex~=2021.20220215.102710 on   │ Warning   │
│                        │ pypi.org, currently only packages from pypi.org are         │           │
│                        │ supported. Custom conda channels are supported if your      │           │
│                        │ package is available there instead.                         │           │
│   powershell_shortcut  │ powershell_shortcut~=0.0.1 has no install candidate for     │ Warning   │
│                        │ linux-64 on pkgs/main                                       │           │
│   morecantile          │ Pip check had the following issues that need resolving:     │ Warning   │
│                        │ morecantile 4.3.0 has requirement pydantic~=1.0, but you    │           │
│                        │ have pydantic 2.0.3.                                        │           │
│   jupyter-server       │ Pip check had the following issues that need resolving:     │ Warning   │
│                        │ jupyter-server 1.13.5 has requirement pywinpty<2; os_name   │           │
│                        │ == "nt", but you have pywinpty 2.0.2.                       │           │
│   icc_rt               │ icc_rt~=2019.0.0 has no install candidate for linux-64 on   │ Warning   │
│                        │ pkgs/main                                                   │           │
│   flask                │ Pip check had the following issues that need resolving:     │ Warning   │
│                        │ flask 2.3.2 has requirement Jinja2>=3.1.2, but you have     │           │
│                        │ jinja2 2.11.3.                                              │           │
│   autopep8             │ Pip check had the following issues that need resolving:     │ Warning   │
│                        │ autopep8 1.6.0 has requirement pycodestyle>=2.8.0, but you  │           │
│                        │ have pycodestyle 2.7.0.                                     │           │
│   TBB                  │ Cannot find TBB~=0.2.0 on pypi.org, currently only packages │ Warning   │
│                        │ from pypi.org are supported. Custom conda channels are      │           │
│                        │ supported if your package is available there instead.       │           │
│                        ╵                                                             ╵           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

ClusterCreationError: Cluster was waiting for 2 workers but 3 (of 4) workers have already failed. You could try requesting fewer workers or adjust `wait_for_workers` if fewer workers would be acceptable.

Failure Reasons
---------------

Software build failed -> Build timed out
	(error affected 3 workers) (cluster_id: 326971)

# Calculate indicators - AOI level

## Compute indicators: One city * One indicator

### HEA-3 Built land with Low Surface reflectivity

In [ ]:
%%time

city_gdf["built_land_with_low_surface_reflectivity"] = built_land_with_low_surface_reflectivity(city_gdf) 
city_gdf

### LND-2 Land with Tree cover

In [9]:
%%time

city_gdf["mean_tree_cover"] = mean_tree_cover(city_gdf) 
city_gdf

/usr/local/anaconda3/envs/cities-cif/lib/python3.10/site-packages/xee/ext.py:549: UserWarning: Unable to retrieve 'system:time_start' values from an ImageCollection due to: No 'system:time_start' values found in the 'ImageCollection'.
  warnings.warn(


CPU times: user 6.76 s, sys: 1.09 s, total: 7.85 s
Wall time: 11.8 s


,id,geo_id,geo_name,geo_level,geo_parent_name,geo_version,geometry,mean_tree_cover
0,0,BRA-Salvador_ADM4-union_1,BRA-Salvador,ADM4union,BRA-Salvador,0,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",28.366773


In [21]:
import ee
import geemap

WC = ee.ImageCollection("ESA/WorldCover/v100")
WorldCover = WC.first();
builtup = WorldCover.eq(50)

## read Trees in Mosaic Landscapes tree cover dataset
TML = ee.ImageCollection('projects/wri-datalab/TML')
TreeCoverImg = TML.reduce(ee.Reducer.mean()).rename('b1')

# https://gis.stackexchange.com/questions/421422/google-earth-engine-image-no-data-mask
TreeDataMask = TreeCoverImg.unmask(-99).neq(-99)   # give no data pixels an abitrary value outside the range of the data. Then make a boolean raster using the assigned value.

fc = geemap.geopandas_to_ee(city_gdf)
pixelcounts = TreeCoverImg.reduceRegions(fc,ee.Reducer.mean(),10)
df = geemap.ee_to_pandas(pixelcounts)
df

,built_land_with_low_surface_reflectivity,geo_id,geo_level,geo_name,geo_parent_name,geo_version,id,mean
0,0.240654,BRA-Salvador_ADM4-union_1,ADM4union,BRA-Salvador,BRA-Salvador,0,0,28.349677


### HEA-4 Percent of Built land without tree cover

In [10]:
%%time

city_gdf["built_land_without_tree_cover"] = built_land_without_tree_cover(city_gdf) 
city_gdf

/usr/local/anaconda3/envs/cities-cif/lib/python3.10/site-packages/xee/ext.py:549: UserWarning: Unable to retrieve 'system:time_start' values from an ImageCollection due to: No 'system:time_start' values found in the 'ImageCollection'.
  warnings.warn(


CPU times: user 8.43 s, sys: 1.63 s, total: 10.1 s
Wall time: 26.2 s


,id,geo_id,geo_name,geo_level,geo_parent_name,geo_version,geometry,mean_tree_cover,built_land_without_tree_cover
0,0,BRA-Salvador_ADM4-union_1,BRA-Salvador,ADM4union,BRA-Salvador,0,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",28.366773,0.848291


### HEA-2 Built Land With High LST

In [11]:
%%time

city_gdf["built_land_with_high_land_surface_temperature"] = built_land_with_high_land_surface_temperature(city_gdf) 
city_gdf

CPU times: user 23.9 s, sys: 44.4 s, total: 1min 8s
Wall time: 1min 10s


,id,geo_id,geo_name,geo_level,geo_parent_name,geo_version,geometry,mean_tree_cover,built_land_without_tree_cover,built_land_with_high_land_surface_temperature
0,0,BRA-Salvador_ADM4-union_1,BRA-Salvador,ADM4union,BRA-Salvador,0,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",28.366773,0.848291,0.080262


## Compute indicators: One city * Multiple indicators

In [40]:
%%time

indicators =get_indicators(city_gdf, indicators=[Indicator.TREE_COVER,
                                                 Indicator.BUILT_LAND_WITH_TREE_COVER])
indicators_df = pd.merge(indicators[0], indicators[1])
indicators_df

CPU times: total: 3.31 s
Wall time: 8.63 s


,id,geo_id,geo_name,geo_level,geo_parent_name,geo_version,geometry,LND_2_percentTreeCover,HEA_4_percentBuiltupWithoutTreeCover
0,0,BRA-Salvador_ADM4-union_1,BRA-Salvador,ADM4union,BRA-Salvador,0,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",0.283478,0.907352


# Caculate indicators - Subcity levels

# Calculate indicators for multiple cities

## Admin Boundary

## Esa World Cover

## Albedo

## Trpical Tree cover

## LST